## Exercise 3
Plot the steady state distributions for different coolings strengths. How does the mean particle energy change with respect to the cooling? Highlight any high-energy power-law tails in the distribution and/or try to fit a Maxwellian distribution to the low-energy part.

In [6]:
from wrapper_sim import Simulation
import pusher
import numpy as np
from matplotlib import figure, axes, patches, lines, colors
from typing import Optional
from warnings import warn
import matplotlib.pyplot as plt
from maxwell_juttner import MaxwellJuttner

In [7]:
particles = int(1e5)
gd = np.geomspace(1,1e3,4)
cs = [{"syn": gsyn, "ic":gic} for gsyn in gd for gic in gd]

saves = 10
temp = 1
names = []

for gamma_drag in cs:
    sim = Simulation()
    sim.begin(particles, temp, gamma_drag, saves)
    sim.run()
    name = f"P1e5-T1-S10-syn{gamma_drag['syn']}-ic{gamma_drag['ic']}"
    sim.end(name)
    names.append(name)

In [ ]:
def axhist_steady_energy_spectrum(ax: axes.Axes, U: np.ndarray, color: str, *args, **kwargs):
    lor = pusher.lorentz_factor(U[-1])
    l = ax.hist(np.log10(lor - 1), histtype="step", bins="fd", color=color, *args, **kwargs)
    return ax, l

def power_law(U: np.ndarray, lorentz_range: Optional[tuple] = None):
    lor = pusher.lorentz_factor(U[-1])
    if lorentz_range is None:
        lorentz_range = (1, np.inf)
    lo,hi = lorentz_range
    temp = MaxwellJuttner.fit(lor[(lo < lor) *  (lor < hi)])
    print(f'fitted temperature: {temp}')
    return temp

def axplot_powerlaw(ax: axes.Axes, temp: float, lorentz_range: tuple, color: str, *args, **kwargs):
    gamma_space = np.geomspace(*lorentz_range, 200)
    mj = MaxwellJuttner()
    mj_space = mj.pdf(gamma_space, temp)
    log_gminus1 = np.log10(gamma_space-1)
    l = ax.plot(log_gminus1, mj_space, color=color, *args, **kwargs)
    return ax, l, np.mean(log_gminus1)

def exercise3(names: list[str]):
    sim = Simulation()

    fig = plt.figure()
    ax = fig.add_subplot()

    handles = []
    labels = []
    for name, color in zip(names, colors.TABLEAU_COLORS.values()):
        sim.load(name)

        ax, l, mean = axhist_steady_energy_spectrum(ax, sim.vel_history, color)
    
        handles.append(l)
        labels.append(fr"$\gamma_\mathrm{{syn}} = {sim.gamma_drag['syn']}, \gamma_\mathrm{{IC}} = {sim.gamma_drag['ic']}, \overline{{\log_{{10}}(\gamma - 1)}} = {mean}$")
    
    ax.set_xlabel(r"$\log_{10}\left(\gamma - 1\right)$")
    ax.set_ylabel(r"$dN_\mathrm{e}/d(\gamma - 1)$")
    ax.set_yscale("log")

    fig.legend(handles, labels)

    return fig, ax